# ECE284 SP25 Project

## Part 1 Install libraries

In [ ]:
!pip install datasets

## Part 2 Import libraries

In [2]:
# Python built-in libraries
import os

# Hugging face libraries
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)

# Other libraries

## Part 3 Set global parameters

In [3]:
# Baseline model
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# Datasets
dataset_name = "GBaker/MedQA-USMLE-4-options"

# Output dir


## Part 4 Load dataset

In [8]:
# Define preprocess function
def preprocess(data):
    '''
        Preprocess dataset for training and validation.
        Each data is a dict.
    '''

    # Pick question, options and answers from data
    question = data["question"]
    answer = data["answer_idx"]

    options = [key + ". " + val for key, val in data["options"].items()]
    options = "\n".join(options)

    # Concatenate information
    instruction = "Please answering the following question "        \
                    "by selecting the correct answer.\n\n"          \
                    f"Question:\n {question}\n\n"                   \
                    f"Options: {options}\n\n"                       \
                    "Provide only the letter of the correct answer."

    # Add prompt format
    instruction_formatted = "<|im_start|>user\n"                    \
                            f"{instruction} <|im_end|>\n"           \
                            "<|im_start|>assistant\n"               \
                            f"{answer} <|im_end|>\n"

    return {"text": instruction_formatted}

In [15]:
# Load dataset
dataset = load_dataset(dataset_name)
column_names = dataset["train"].column_names
dataset = dataset.map(preprocess, remove_columns=column_names)

train_dataset = dataset["train"]
test_dataset = dataset["test"]

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0]

In [1]:
x = "abc"
y = "456"
z = "def"

a = f"test:{x}" \
    "test2:{y}" \
    "test3:{z}"

print(a)

test:abctest2:{y}test3:{z}


## Part 5 Load model

## Part 6 Fine-tune